# Different Bounce Thresholds

In [1]:
import sympy
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Video, HTML, update_display
import time
from typing import Literal
from functools import partial

import sys
sys.path += ["../src", "./src"]

import solvers
import geometry
import animation

In [2]:
sympy.var("x y", real=True)

curve = geometry.CurveConstraint(x**4 + y**2 - 2*x**2 +2*x**3 - 1, x, y, inside="negative", xlim=(-3,1.2))

In [3]:
gravity = lambda q, v: np.array([0, -3])

# different bounce thresholds
ball1 = geometry.Ball(curve, mass=1., gravity=gravity, curve_friction=0.5, normal_cor=0.5, bounce_threshold=0, name="threshold 0")
ball2 = geometry.Ball(curve, mass=1., gravity=gravity, curve_friction=0.5, normal_cor=0.5, bounce_threshold=0.1, name="threshold 0.1")
ball3 = geometry.Ball(curve, mass=1., gravity=gravity, curve_friction=0.5, normal_cor=0.5, bounce_threshold=3, name="threshold 3")

In [4]:
t_end = 8
steps = int(100*t_end)

status = widgets.HTML("-")
display(status)

anim = animation.BallAnimation(curve, [ball1, ball2, ball3], length=steps)

def callback(step, qnew, vnew, error, mode: Literal["dae", "ode"], ball_id):
    
    status.value = F'<b>step {step} of {steps}</b> <progress value="{step}" max="{steps}"> </progress>'
    anim.update(qnew, vnew, ball_id, step-1, mode)

ball1.simulate(steps, t_end, q0=(1, -1), v_up=3, callback=partial(callback, ball_id=0)) # also interesting: (-3, 5), 0; (4,5), -1; (2,2), -1; (0,5), 0
ball2.simulate(steps, t_end, q0=(1, -1), v_up=3, callback=partial(callback, ball_id=1))
ball3.simulate(steps, t_end, q0=(1, -1), v_up=3, callback=partial(callback, ball_id=2))

HTML(value='-')

In [5]:
# display
HTML(anim.render(1/60, show_trace=True, trace_alpha=0.4))
# anim.render(1/60, filename="_static/bounce_threshold.mp4", show_trace=True, trace_alpha=0.4)

/home/norberth/venveria/venvs/sc/lib/python3.13/site-packages/matplotlib/cbook.py:1719: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/home/norberth/venveria/venvs/sc/lib/python3.13/site-packages/matplotlib/cbook.py:1355: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


HTML(value='-')

The above showcases three balls that switch back to the rolling phase when the damped normal velocity after an impact lies below 0, 0.1 and 3, respectively.
As one can see, the ball that cannot switch back to rolling behaves similarly to the ball with threshold 0.1. The ball with threshold 3 does not bounce but sticks to the curve too easily.
When in doubt, it may therefore be advisable to set a lower bounce threshold.